# Your first deep neural network

# imports

In [ ]:
import torch
import torch.nn as nn
from torch.nn.functional import one_hot
from torchvision.transforms import ToTensor
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

# data

In [ ]:
NUM_CLASSES = 10

In [ ]:
train_data = CIFAR10('~/pytorch', train=True, transform=ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=32)
test_data = CIFAR10('~/pytorch', train=False, transform=ToTensor(), download=True)
test_loader = DataLoader(test_data, batch_size=32)

# architecture

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 200),
    nn.ReLU(),
    nn.Linear(200, 150),
    nn.ReLU(),
    nn.Linear(150, NUM_CLASSES),
    nn.Softmax(dim=0))

In [ ]:
print(model)

# train

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = torch.nn.BCELoss(reduction='sum')
for t in range(10):
    print("Epoch {}/10".format(t + 1))
    total_loss = 0
    for x, y in tqdm(train_loader):
        output = model(x)
        y_pred = one_hot(y, NUM_CLASSES).float()
        loss = criterion(output, y_pred)
        total_loss += loss.item()
        model.zero_grad()
        loss.backward()
        optimizer.step()
    print("Loss: {:4f}".format(total_loss / len(train_data)))

# analysis

In [ ]:
model.eval()
correct = 0
x = 0.0
y = 1.0
for test_x, test_y in tqdm(test_loader):
    output = model(test_x)
    y_pred = output.argmax(dim=1)
    correct += (y_pred == test_y).float().sum()
print("Accuracy: {:.4f}".format(correct / len(test_data)))